In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
import time

# ---------------------- Load Data ----------------------
column_names = ['engine_id', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]
train_df = pd.read_csv("train_FD001.txt", sep="\s+", header=None, names=column_names)
test_df = pd.read_csv("test_FD001.txt", sep="\s+", header=None, names=column_names)
rul_df = pd.read_csv("RUL_FD001.txt", header=None, names=["true_RUL"])
rul_df["engine_id"] = rul_df.index + 1

# Add RUL to train set
max_cycle = train_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle.columns = ['engine_id', 'max_cycle']
train_df = train_df.merge(max_cycle, on='engine_id')
train_df['RUL'] = train_df['max_cycle'] - train_df['cycle']
train_df.drop('max_cycle', axis=1, inplace=True)

# Add RUL to test set
test_max_cycle = test_df.groupby('engine_id')['cycle'].max().reset_index()
test_max_cycle.columns = ['engine_id', 'max_cycle']
test_df = test_df.merge(test_max_cycle, on='engine_id')
test_df['RUL'] = test_df['max_cycle'] - test_df['cycle']
test_df.drop('max_cycle', axis=1, inplace=True)

# Clip RUL (optional but common)
rul_cap = 130
train_df['RUL'] = train_df['RUL'].clip(upper=rul_cap)
test_df['RUL'] = test_df['RUL'].clip(upper=rul_cap)

# ---------------------- Preprocess ----------------------
features = train_df.columns.difference(['engine_id', 'cycle', 'RUL', 'sensor_22', 'sensor_23'])

scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# ---------------------- Sequence Creator ----------------------
sequence_length = 30

def create_sequences(df, sequence_length, features):
    sequences, labels = [], []
    for engine_id in df['engine_id'].unique():
        engine_data = df[df['engine_id'] == engine_id]
        for i in range(len(engine_data) - sequence_length):
            seq = engine_data[features].iloc[i:i+sequence_length].values
            label = engine_data['RUL'].iloc[i + sequence_length]
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels)

X_train_seq, y_train_seq = create_sequences(train_df, sequence_length, features)
num_features = X_train_seq.shape[2]

# ---------------------- Model ----------------------
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(sequence_length, num_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=['mae'])
model.summary()

# ---------------------- Training ----------------------
start_time = time.time()
model.fit(X_train_seq, y_train_seq, validation_split=0.25, epochs=100, batch_size=64)
print(f"Training Time: {time.time() - start_time:.2f} seconds")

# ---------------------- Test Evaluation ----------------------
# Only take the last sequence per engine
test_sequences = []
valid_engine_ids = []

for engine_id in test_df["engine_id"].unique():
    engine_data = test_df[test_df["engine_id"] == engine_id]
    if len(engine_data) >= sequence_length:
        last_seq = engine_data[features].iloc[-sequence_length:].values
        test_sequences.append(last_seq)
        valid_engine_ids.append(engine_id)

X_test_final = np.array(test_sequences)
print("X_test_final shape before predict:", X_test_final.shape)
print("Model input shape:", model.input_shape)
y_pred = model.predict(X_test_final).flatten()
print("y_pred shape:", y_pred.shape)  # Should be (100,)
y_true = rul_df[rul_df["engine_id"].isin(valid_engine_ids)]["true_RUL"].values


# ---------------------- Evaluation ----------------------
def evaluate_model(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    h = y_pred - y_true
    score = np.sum(np.where(h < 0, np.exp(-h / 13) - 1, np.exp(h / 10) - 1))
    print(f"{name}: RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.2f}, Accuracy={accuracy:.2f}%, Score={score:.2f}")

evaluate_model("CNN-BiLSTM Hybrid Model", y_true, y_pred)


C:\Users\D-InfoTech\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)                    │ (None, 28, 128)             │           9,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (None, 14, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ (None, 14, 256)             │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 14, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 436,993 (1.67 MB)

 Trainable params: 436,993 (1.67 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - loss: 7463.6064 - mae: 74.9420 - val_loss: 6777.9517 - val_mae: 71.2600
Epoch 2/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 5918.5820 - mae: 65.3532 - val_loss: 6327.2603 - val_mae: 68.5654
Epoch 3/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 5541.4087 - mae: 63.0430 - val_loss: 5961.5991 - val_mae: 66.3716
Epoch 4/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 5293.0210 - mae: 61.5527 - val_loss: 5633.7065 - val_mae: 64.3949
Epoch 5/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 4936.1074 - mae: 59.2550 - val_loss: 5330.9233 - val_mae: 62.5575
Epoch 6/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - loss: 4670.5449 - mae: 57.6252 - val_loss: 5037.9668 - val_mae: 60.5644
Epoch 7/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - loss: 4316.1597 - mae: 54.6265 - val_loss: 4731.4370 - val_mae: 57.6572
Epoch 8/100
207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 4058.1985 - mae: 51.9365 - val_loss: 4452

In [10]:
print (y_pred)

[119.398476  124.31119    43.447796  125.05897    86.02975   108.214355
 120.843185  120.520874  128.01208    92.74121    89.56775   105.34804
  82.15187   100.80336    96.701805   99.983215   56.065754   31.997206
  88.941376   14.772558   89.53344   129.12611   129.11362    18.747961
 109.85289   105.32471   103.99496    89.19165    86.168015  100.08197
   9.0746975  55.34666   128.24188     5.8481665  17.22873    22.01872
  37.78433    65.62877   128.15746    33.390106   30.277021   10.955322
  84.25946   127.78528    98.32408    31.82313   124.60323   105.86696
  15.586316  126.94974    91.82101    30.255625   30.506048  128.47101
 124.80773    16.525417   96.35958    54.18444   115.92589   101.50987
  20.648531   56.09933    94.16754    29.026627  128.67621    15.696024
 128.3964     11.418528  128.83833    92.041245  121.03865    80.035385
  97.865746   91.77104   111.15451     7.412954   34.312508  129.42343
  89.36007    98.41931     6.796429    9.791659  128.9452     90.0833
 